In [ ]:
!pip install huggingsound

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel("Akashpb13/Swahili_xlsr")
audio_paths = ["audio_00b84616.mp3", "audio_0a0f9147.mp3"]

transcriptions = model.transcribe(audio_paths)

print(transcriptions)



INFO:huggingsound.speech_recognition.model:Loading model...
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 2/2 [00:08<00:00,  4.22s/it]

[{'transcription': 'atakuwa na mambo mawili ya kukabiliana nayo', 'start_timestamps': [520, 600, 640, 780, 820, 900, 940, 980, 1060, 1080, 1100, 1200, 1260, 1360, 1380, 1420, 1460, 1540, 1580, 1680, 1740, 1860, 1900, 1920, 1960, 2000, 2020, 2100, 2120, 2220, 2260, 2380, 2400, 2520, 2540, 2640, 2740, 2760, 2800, 2880, 2940, 3100, 3140], 'end_timestamps': [540, 620, 660, 800, 840, 920, 960, 1000, 1080, 1100, 1120, 1220, 1280, 1380, 1400, 1440, 1480, 1560, 1600, 1700, 1760, 1880, 1920, 1940, 1980, 2020, 2040, 2120, 2140, 2240, 2280, 2400, 2420, 2540, 2560, 2660, 2760, 2780, 2820, 2900, 2960, 3120, 3160], 'probabilities': [0.998897910118103, 0.9996330738067627, 0.999882698059082, 0.9997827410697937, 0.999816358089447, 0.9944215416908264, 0.9999090433120728, 0.9998611211776733, 0.9997653365135193, 0.9999184608459473, 0.8238128423690796, 0.999818742275238, 0.9998225569725037, 0.9998531341552734, 0.9854356646537781, 0.9998458623886108, 0.999919056892395, 0.9998569488525391, 0.9998936653137207

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/

/content/drive


In [ ]:
cd MyDrive/

/content/drive/MyDrive


In [ ]:
cd MachineLearning/

/content/drive/MyDrive/MachineLearning


In [ ]:
cd Data/

/content/drive/MyDrive/MachineLearning/Data


In [ ]:
cd ASR challenge

/content/drive/MyDrive/MachineLearning/Data/ASR challenge


In [ ]:
ls

colab_book.ipynb      test_audios.zip       train_wavs/
notebook1.ipynb       test.csv              Untitled0.ipynb
notebook2.ipynb       train/                Untitledagain.ipynb
notebook3.ipynb       train_audios/         VariableDefinitions.csv
SampleSubmission.csv  train_audios.zip      vocab.json
submission5.csv       train.csv             wav2vector.ipynb
test/                 trained_asr_model.h5  wavs/
test_audios/          TrainedModel.h5


In [ ]:
train_path = 'train_audios/train_audios/'
metadata_path =  "train.csv"

import pandas as pd
# Read metadata file and parse it
metadata_df = pd.read_csv(metadata_path)
metadata_df = metadata_df.sample(frac=1).reset_index(drop=True)
metadata_df.head(3)

,audio_ID,path,sentence
0,audio_3e7df4e9,audio_3e7df4e9.mp3,Tarime Rorya hao ni Wakurya wa kaskazini Mara
1,audio_0596ad2a,audio_0596ad2a.mp3,li hili ningependa nikuulize bukuku
2,audio_7bde792a,audio_7bde792a.mp3,ambayo yamepangwa utekelezaji wake kukamilika ...


In [ ]:
split = int(len(metadata_df) * 0.1)
split_= int(len(metadata_df) * 0.98)
df_train = metadata_df[:split]
df_val = metadata_df[split_:]

print(f"Size of the training set: {len(df_train)}")
print(f"Size of the training set: {len(df_val)}")

Size of the training set: 1909
Size of the training set: 382


In [ ]:
batch_size = 32
# Define the training dataset
import tensorflow as tf
train_data = tf.data.Dataset.from_tensor_slices(
    (list(df_train["path"]), list(df_train["sentence"]))
)
# train_data = (
#     train_data.map(num_parallel_calls=tf.data.AUTOTUNE)
#     .padded_batch(batch_size)
#     .prefetch(buffer_size=tf.data.AUTOTUNE)
# )

# Define the validation dataset
eval_data = tf.data.Dataset.from_tensor_slices(
    (list(df_val["path"]), list(df_val["sentence"]))
)
# eval_data = (
#     eval_data.map(num_parallel_calls=tf.data.AUTOTUNE)
#     .padded_batch(batch_size)
#     .prefetch(buffer_size=tf.data.AUTOTUNE)
# )

In [ ]:
from huggingsound import TrainingArguments, ModelArguments, SpeechRecognitionModel, TokenSet

model = SpeechRecognitionModel("Akashpb13/Swahili_xlsr")
output_dir = "my/finetuned/model/output/dir"

# first of all, you need to define your model's token set
# however, the token set is only needed for non-finetuned models
# if you pass a new token set for an already finetuned model, it'll be ignored during training
tokens = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z", "'"]
token_set = TokenSet(tokens)


# and finally, fine-tune your model
model.finetune(
    output_dir,
    train_data=train_data,
    eval_data=eval_data, # the eval_data is optional
    token_set=token_set,
)


INFO:huggingsound.speech_recognition.model:Loading model...


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:53: FutureWarning: Loading a tokenizer inside Wav2Vec2Processor from a config that does not include a `tokenizer_class` attribute is deprecated and will be removed in v5. Please add `'tokenizer_class': 'Wav2Vec2CTCTokenizer'` attribute to either your `config.json` or `tokenizer_config.json` file to suppress this warning: 
  warnings.warn(
`use_fast` is set to `True` but the tokenizer class does not have a fast version.  Falling back to the slow version.
INFO:huggingsound.speech_recognition.model:Loading training data...
INFO:huggingsound.speech_recognition.model:Converting data format...
INFO:huggingsound.speech_recognition.model:Preparing data input and labels...


Map:   0%|          | 0/17184 [00:00<?, ? examples/s]